In [2]:
import numpy as np
import pandas as pd
from graphviz import Digraph
import sys


In [5]:
df = pd.read_csv('../ressources/tennis.csv')


In [9]:
df2 = pd.read_csv('../ressources/mushrooms.csv')

In [10]:
class Tree:
    def __init__(self):
        self.sons = {}
        self.Attribute = None
    def add_son(self, son, att):
        self.sons[att] = son
    def set_att(self, att):
        self.Attribute = att
    def genView(self, dot):
        label = self.Attribute
        dot.node(str(self), label)
        for key, son in self.sons.items():
            dot.edge(str(self), str(son), label=key)
            son.genView(dot)
   

In [11]:
def get_entropy(data,target):
    n = len(data)
    acc = 0
    for c  in data[target].value_counts():
        acc -= c/n * np.log2(c/n)
    return acc
        
    
    

In [12]:
def ID3(data, target, func_ent):
    root = Tree()
    if (len(set(data[target])) == 1 ):
        root.set_att(data[target].values[0])     
    else:
        lis_ent = []
        for elt in data:
            if(elt == target):
                lis_ent.append(np.inf)
                continue
            ent = 0
            l = [x for _,x in data.groupby(elt)]
            for d in l:
                ent += func_ent(d, target)
            lis_ent.append(ent)
            indop = np.argmin(lis_ent)        
            optimal = data.columns[indop]
            root.set_att(optimal)
        for d in [(att, x) for att,x in data.groupby(optimal)]:
            root.add_son(ID3(d[1].loc[:, d[1].columns != optimal], target, func_ent), d[0])
    return root

In [13]:
target = 'PlayTennis'
tree = ID3(df, target, get_entropy)
dot = Digraph(comment="Decision tree")
tree.genView(dot)
dot.view(cleanup=True)

'Digraph.gv.pdf'

In [14]:
target2 = 'EDIBLE'
tree2 = ID3(df2, target2, get_entropy)
dot = Digraph(comment="Decision tree")
tree2.genView(dot)
dot.view(cleanup=True)

'Digraph.gv.pdf'